In [ ]:
import os
import pyodbc
import pandas as pd
import numpy as np
import shutil

In [ ]:
server = 'bo105'
database = 'test_db'
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

In [ ]:
UnProcessedPath = os.path.join('C:', os.sep, 'FilePath','UnProcessed')
ProcessedPath = os.path.join('C:', os.sep, 'FilePath','Processed')

unprocessedfilepath = os.path.join(UnProcessedPath, xlsfile)
processedfilepath = os.path.join(ProcessedPath, xlsfile)

In [ ]:
with os.scandir(UnProcessedPath) as files:
    for file in files:
        if file.is_file():
            xlsfile=file.name

In [ ]:
df_xlsx = pd.read_excel(unprocessedfilepath)
cleandf = df_xlsx.dropna(how='all')

In [ ]:
cleandf['Truck ID']=cleandf['Truck ID'].astype(str)
cleandf['Invoice_Date']=cleandf['Invoice_Date'].astype(str)
cleandf['PrePass_Id']=cleandf['PrePass_Id'].astype(str)
cleandf['Entry_Date']=cleandf['Entry_Date'].astype(str)
cleandf['Class']=cleandf['Class'].astype(str)
cleandf['Miles']=cleandf['Miles'].astype(str)
cleandf['Toll_Charge']=cleandf['Toll_Charge'].astype(str)

In [ ]:
for index, row in cleandf.iterrows():
    cursor.execute("INSERT INTO [dbo].[db_test]([First five],[No space],[Correct unit#],[Truck Status],[Division],[Truck Class],[Truck ID],[Post_Date],[Invoice_Date],[Source],[Read_Type],[PrePass_Id],[EZPassId_Plate],[Truck_Id],[Agency],[Entry_Plaza],[Entry_Date],[Exit_Plaza],[Exit_Date],[Class],[Miles],[Toll_Charge]) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row['First five'], row['No space'], row['Correct unit#'], row['Truck Status'], row['Division'], row['Truck Class'], row['Truck ID'], row['Post_Date'], row['Invoice_Date'], row['Source'], row['Read_Type'], row['PrePass_Id'], row['EZPassId_Plate'], row['Truck_Id'], row['Agency'], row['Entry_Plaza'], row['Entry_Date'], row['Exit_Plaza'], row['Exit_Date'], row['Class'], row['Miles'], row['Toll_Charge'])

conn.commit()
cursor.close()
conn.close()

In [ ]:
shutil.move(unprocessedfilepath, processedfilepath)